# Sequence를 처리하기 위한 RNN 

1. 주어진 데이터를 RNN에 넣을 수 있는 형태로 만듭니다.
2. 기본적인 RNN 사용법 및 적용법을 익힙니다.
3. LSTM, GRU의 사용법 및 적용법을 익힙니다.

In [ ]:
from tqdm import tqdm
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch

## 데이터 전처리

아래의 sample data를 확인해봅시다.  
전체 단어 수와 pad token의 id도 아래와 같습니다.

In [ ]:
vocab_size = 100
pad_id = 0

data = [
  [85,14,80,34,99,20,31,65,53,86,3,58,30,4,11,6,50,71,74,13],
  [62,76,79,66,32],
  [93,77,16,67,46,74,24,70],
  [19,83,88,22,57,40,75,82,4,46],
  [70,28,30,24,76,84,92,76,77,51,7,20,82,94,57],
  [58,13,40,61,88,18,92,89,8,14,61,67,49,59,45,12,47,5],
  [22,5,21,84,39,6,9,84,36,59,32,30,69,70,82,56,1],
  [94,21,79,24,3,86],
  [80,80,33,63,34,63],
  [87,32,79,65,2,96,43,80,85,20,41,52,95,50,35,96,24,80]
]

In [ ]:
data[0]

Padding 처리를 해주면서 padding 전 길이도 저장합니다.

In [ ]:
max_len = len(max(data, key=len))
print(f"Maximum sequence length: {max_len}")

valid_lens = []
for i, seq in enumerate(tqdm(data)):
  valid_lens.append(len(seq))
  if len(seq) < max_len:
    data[i] = seq + [pad_id] * (max_len - len(seq))

In [ ]:
print(data)
print(valid_lens)

In [ ]:
# B: batch size, L: maximum sequence length
batch = torch.LongTensor(data)  # (B, L)
batch_lens = torch.LongTensor(valid_lens)  # (B)

In [ ]:
batch.shape

In [ ]:
batch_lens, sorted_idx = batch_lens.sort(descending=True)
batch = batch[sorted_idx]

In [ ]:
print(batch)
print(batch_lens)

## RNN 사용해보기

RNN에 넣기 전 word embedding을 위한 embedding layer를 만듭니다.

In [ ]:
embedding_size = 256

## TODO ##
embedding = None

# d_w: embedding size
batch_emb = embedding(batch)  # (B, L, d_w)

In [ ]:
batch.shape

In [ ]:
batch_emb.shape

RNN 모델 및 초기 hidden state를 정의

- batch_emb 변수를 RNN에 넣을 예정입니다.
- torch 공식 문서를 참조하여, RNN 모델을 정의해보세요. 
- input size는 어떻게 되어야 하나요?

In [ ]:
hidden_size = 512  # RNN의 hidden size
num_layers = 1  # 쌓을 RNN layer의 개수
num_dirs = 1  # 1: 단방향 RNN, 2: 양방향 RNN

rnn = nn.RNN(
    # TODO #
    input_size = None,
    hidden_size = hidden_size,
    num_layers = num_layers,
    bidirectional = True if num_dirs > 1 else False,
    batch_first = None
)

h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h)

### **Vanilla RNN 활용법**

RNN에 batch data를 넣으면 아래와 같이 2가지 output을 얻습니다.


*   `hidden_states`: 각 time step에 해당하는 hidden state들의 묶음.
*   `h_n`: 모든 sequence를 거치고 나온 마지막 hidden state.

torch의 RNN 문서를 참조하여서, ``batch_emb``변수를 rnn에 input으로 넣어보세요.
나온 결과의 shape도 출력해보세요. 

마지막 hidden state를 이용하여 text classification task에 적용할 수 있습니다.

In [ ]:
num_classes = 2
classification_layer = nn.Linear(hidden_size, num_classes)

# C: number of classes
output = classification_layer(h_n.squeeze(0))  # (1, B, d_h) => (B, C)
print(output.shape)

각 time step에 대한 hidden state를 이용하여 token-level의 task를 수행할 수도 있습니다.

In [ ]:
num_classes = 5
entity_layer = nn.Linear(hidden_size, num_classes)

# C: number of classes
output = entity_layer(hidden_states)  # (L, B, d_h) => (L, B, C)
print(output.shape)

### **LSTM 활용법**

LSTM에선 cell state가 추가됩니다.  
Cell state의 shape는 hidden state의 그것과 동일합니다.

- batch_emb 변수를 LSTM에 넣을 예정입니다.
- torch 공식 문서를 참조하여, LSTM 모델을 정의해보세요. 
- input size는 어떻게 되어야 하나요?

In [ ]:
hidden_size = 512
num_layers = 1
num_dirs = 1

## TODO ##
lstm = nn.LSTM(
    input_size = None,
    hidden_size = None,
    num_layers = num_layers,
    bidirectional = True if num_dirs > 1 else False,
    batch_first = None
)

h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h)
c_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h)

torch의 LSTM 문서를 참조하여서, ``batch_emb``변수를 rnn에 input으로 넣어보세요.
나온 결과의 shape도 출력해보세요. 

In [ ]:
## TODO ##

### **GRU 사용**

GRU는 cell state가 없어 RNN과 동일하게 사용 가능합니다.   
GRU를 이용하여 LM task를 수행해봅시다.

In [ ]:
gru = nn.GRU(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=True if num_dirs > 1 else False
)

In [ ]:
output_layer = nn.Linear(hidden_size, vocab_size)

In [ ]:
input_id = batch.transpose(0, 1)[0, :]  # (B)
hidden = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (1, B, d_h)

In [ ]:
for t in range(max_len):
  input_emb = embedding(input_id).unsqueeze(0)  # (1, B, d_w)
  output, hidden = gru(input_emb, hidden)  # output: (1, B, d_h), hidden: (1, B, d_h)

  # V: vocab size
  output = output_layer(output)  # (1, B, V)
  probs, top_id = torch.max(output, dim=-1)  # probs: (1, B), top_id: (1, B)

  print("*" * 50)
  print(f"Time step: {t}")
  print(output.shape)
  print(probs.shape)
  print(top_id.shape)

  input_id = top_id.squeeze(0)  # (B)

### **양방향 및 여러 layer 사용**

이번엔 양방향 + 2개 이상의 layer를 쓸 때 얻을 수 있는 결과에 대해 알아봅니다.


In [ ]:
num_layers = 2
num_dirs = 2
dropout=0.1

gru = nn.GRU(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    dropout=dropout,
    bidirectional=True if num_dirs > 1 else False
)

Bidirectional이 되었고 layer의 개수가 $2$로 늘었기 때문에 hidden state의 shape도 `(4, B, d_h)`가 됩니다.

In [ ]:
# d_w: word embedding size, num_layers: layer의 개수, num_dirs: 방향의 개수
batch_emb = embedding(batch)  # (B, L, d_w)
h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h) = (4, B, d_h)

packed_batch = pack_padded_sequence(batch_emb.transpose(0, 1), batch_lens)

packed_outputs, h_n = gru(packed_batch, h_0)
print(packed_outputs)
print(packed_outputs[0].shape)
print(h_n.shape)

In [ ]:
outputs, output_lens = pad_packed_sequence(packed_outputs)

print(outputs.shape)  # (L, B, num_dirs*d_h)
print(output_lens)

각각의 결과물의 shape는 다음과 같습니다.

`outputs`: `(max_len, batch_size, num_dir * hidden_size)`  
`h_n`: `(num_layers*num_dirs, batch_size, hidden_size)`

In [ ]:
batch_size = h_n.shape[1]
print(h_n.view(num_layers, num_dirs, batch_size, hidden_size))
print(h_n.view(num_layers, num_dirs, batch_size, hidden_size).shape)